In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [3]:
train = pd.read_csv('Train_data.csv')
test = pd.read_csv('Test_data.csv')

In [113]:
# List of all functions:
    
def df_details(df):
    
    print("Shape of df: ", df.shape)
    print("No of cols: ", len(df.columns))
    print("Column names: ", list(df.columns))
    print()
    print("Missing data in columns: ")
    print(df.isna().sum())
    print()
    print(df.describe())
    
    
def extract_hastags(text):   
    return list(re.findall("[#]\w+", text))


def extract_mentions(text):    
    return re.findall("[@]\w+", text)


def extract_urls(text):    
    return [i[0] for i in re.findall(r"(https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256})", text)]

In [23]:
df_details(train)

Shape of df:  (7920, 3)
No of cols:  3
Column names:  ['id', 'label', 'tweet']

Missing data in columns: 
id       0
label    0
tweet    0
dtype: int64

                id        label
count  7920.000000  7920.000000
mean   3960.500000     0.255808
std    2286.451399     0.436342
min       1.000000     0.000000
25%    1980.750000     0.000000
50%    3960.500000     0.000000
75%    5940.250000     1.000000
max    7920.000000     1.000000


In [24]:
df_details(test)

Shape of df:  (1953, 2)
No of cols:  2
Column names:  ['id', 'tweet']

Missing data in columns: 
id       0
tweet    0
dtype: int64

                id
count  1953.000000
mean   8897.000000
std     563.926857
min    7921.000000
25%    8409.000000
50%    8897.000000
75%    9385.000000
max    9873.000000


In [25]:
train['Train_Test'] = 'Train'
test['Train_Test'] = 'Test'

In [26]:
train.head()

,id,label,tweet,Train_Test
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,Train
1,2,0,Finally a transparant silicon case ^^ Thanks t...,Train
2,3,0,We love this! Would you go? #talk #makememorie...,Train
3,4,0,I'm wired I know I'm George I was made that wa...,Train
4,5,1,What amazing service! Apple won't even talk to...,Train


In [28]:
test.head()

,id,tweet,Train_Test
0,7921,I hate the new #iphone upgrade. Won't let me d...,Test
1,7922,currently shitting my fucking pants. #apple #i...,Test
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t...",Test
3,7924,My ipod is officially dead. I lost all my pict...,Test
4,7925,Been fighting iTunes all night! I only want th...,Test


In [30]:
df = train.append(test)

In [31]:
df_details(df)

Shape of df:  (9873, 4)
No of cols:  4
Column names:  ['id', 'label', 'tweet', 'Train_Test']

Missing data in columns: 
id               0
label         1953
tweet            0
Train_Test       0
dtype: int64

                id        label
count  9873.000000  7920.000000
mean   4937.000000     0.255808
std    2850.233938     0.436342
min       1.000000     0.000000
25%    2469.000000     0.000000
50%    4937.000000     0.000000
75%    7405.000000     1.000000
max    9873.000000     1.000000


In [111]:
df['Hastags'] = df.tweet.apply(extract_hastags)
df['User Mentions'] = df.tweet.apply(extract_mentions)
df['URL'] = df.tweet.apply(extract_urls)

In [112]:
df.head()

,id,label,tweet,Train_Test,Hastags,User Mentions,URL
0,1,0.0,#fingerprint #Pregnancy Test https://goo.gl/h1...,Train,"[#fingerprint, #Pregnancy, #android, #apps, #b...",[],[https://goo.gl]
1,2,0.0,Finally a transparant silicon case ^^ Thanks t...,Train,"[#yay, #Sony, #Xperia, #S, #sonyexperias]",[],[http://instagram.com]
2,3,0.0,We love this! Would you go? #talk #makememorie...,Train,"[#talk, #makememories, #unplug, #relax, #iphon...",[],[http://fb.me]
3,4,0.0,I'm wired I know I'm George I was made that wa...,Train,"[#iphone, #cute, #daventry, #home]",[],[http://instagr.am]
4,5,1.0,What amazing service! Apple won't even talk to...,Train,[],[],[]
